## what to do next

class bitmexDataHandler
- bm_ws_market  订阅 成交明细，买一卖一
- snapshot()   500毫秒一个切片，发送MarketEvent

其中bm_ws_market中的如下成员有用：
- prev_bar
- current_bar
- bid1, ask1

todo:
1. utils.calc_ts()
2. 发布订阅 bar_type, bm_ws_market.subscribe('XBTUSD', ['1m', '5m'], false)
3. bar和last_price需要有两个时间戳：交易所时间和本机实际收到的时间
4. 定义一些全局变量，eg. 枚举类 BarType, def n_minutes()获取对应的分钟数

bar_type 可以拼成任何时间周期，eg. 9m, 17m, 18m, 可以有shift   
所以bar_type的格式是这样的： '9m3s', '18m25s'   
trading_day, ts = 20180101, 190015   
目前可以实现的： trading_day, ts = calc_td_ts()， ts保留至秒钟

## DataHandler.py

In [1]:
# class Orderbook(object): 
#     def __init__(self, symbol=None, bid1=None, bid1vol=None, ask1=None, ask1vol=None, timestamp=None):
#         self.symbol = symbol
#         self.bid1 = bid1
#         self.bid1vol = bid1vol
#         self.ask1 = ask1
#         self.ask1vol = ask1vol
#         self.timestamp = timestamp
        
#     def __repr__(self):
#         return self.__dict__.__repr__()
        

# class Tick(object): 
#     def __init__(self, symbol=None, price=None, volume=None, direction=None, timestamp=None):
#         self.symbol = symbol
#         self.price = price
#         self.volume = volume
#         self.direction = direction
#         self.timestamp = timestamp
        
#     def __repr__(self):
#         return self.__dict__.__repr__()
    
# class Bar(object):
#     pass

# class Snapshot(object):
#     pass

In [2]:
# from bitmexWSMarket2 import bitmexWSMarket2
# from bitmexREST import bitmexREST
# from utils import generate_logger, calculate_td_ts
# import queue


# class bitmexDataHandler(object):
#     def __init__(self, g):
#         self.g = g                            # global settings
#         self.symbols = g.symbols              # 订阅的标的  ['XBTUSD', ...]
#         self.market_data_q = queue.Queue()    # MarketData队列（带数据）        
#         self.logger = generate_logger('DataHandler', g.loglevel, g.logfile)  # 日志
        
#     def add_event_q(self, event_q):
#         self.event_q = event_q                # 全局事件队列
        
#     def run(self):
#         self.__construct_bm_ws_market()
        
#         while True:
#             try:
#                 data = self.market_data_q.get(timeout=100)
#             except queue.Empty:
#                 self.logger.warning('no data in market_data_q for 100 seconds')
#             else:
#                 if isinstance(data, Tick):
#                     self.processTick(data)
#                 elif isinstance(data, Orderbook):
#                     self.processOrderbook(data)
#                 else:
#                     self.logger.warning('Invalid data type from market_data_q: %s' % data.__class__)
    
#     def stop(self):
#         self.logger.info('Stopping bitmexDataHandler ...')
#         self.bm_ws_market.exit()
        
#     def __construct_bm_ws_market(self):
#         self.bm_ws_market = bitmexWSMarket2(apiKey=None, apiSecret=None, 
#                                             is_test=self.g.is_test, loglevel=self.g.loglevel, logfile=self.g.logfile)
#         self.bm_ws_market.connect()
#         self.bm_ws_market.add_market_data_q(self.market_data_q)
#         for s in self.symbols:
#             self.bm_ws_market.subscribe(s, trade=True, orderbook=True)
#         self.bm_ws_market.wait_for_data()
        

#     def processTick(self, tick):
#         self.logger.debug('Processing Tick... %s' % tick)
#         self.event_q.put(tick)   # temp, for test
        
#         # 1. 更新last_price
#         self.__update_last_price(tick)
        
#         # 2. if 该symbol订阅了tick事件，推送（全局事件队列）
#         if False:
#             self.__push_tick_event(tick.symbol)
        
#         # 3. 生成bar
#         self.__bar(tick)
        
#     def __update_tick(self, tick):
#         pass
    
#     def __push_tick_event(self):
#         pass
    
#     def __bar(tick):
#         pass
    
#     def processOrderbook(self, ob):
#         self.logger.debug('Processing Orderbook... %s' % ob)
#         self.event_q.put(ob)    # temp, for test
        
#         # 1. 更新Orderbook
#         self.__update_orderbook(ob)
        
#         # 2. if 该symbol订阅了orderbook事件，推送（全局事件队列）
#         if False:
#             self.__push_orderbook_event(tick.symbol)
        
#     def __update_orderbook(self, ob):
#         pass
    
#     def __push_orderbook_event(self, symbol):
#         pass
    
#     def register_bar_event(self, symbol, bar_type):
#         pass
    
#     def snapshot(self, symbol):
#         """参照国内期货快照数据结构"""
#         pass
    
    

## main.py

In [5]:
# # 主程序
# import queue

# # 加载全局设置
# from GlobalSettings import GlobalSettings


# g = GlobalSettings()
# g.from_config_file('global_settings.json')
# g.__dict__

# # 事件队列
# event_q = queue.Queue()

# # DataHandler
# datahandler = bitmexDataHandler(g)
# datahandler.add_event_q(event_q)

# datahandler.run()

# # datahandler.register_bar_event('XBTUSD', '1m')
# # for s,bar_type in params.subscription:
# #     datahandler.register_bar_event(s, bar_type)


# print('===============================================')
# while True:
#     try:
#         a = event_q.get(timeout=10)
#     except queue.Empty:
#         print('warning: no data in 10 sec')
#     else:
#         if isinstance(a, Tick):
#             print('💛 💛 💛 💛 💛   Tick  💛 💛 💛 💛 💛\n %s' % a)
#         elif isinstance(a, Orderbook):
#             print('✡️ ✡️ ✡️ ✡️ ✡️   Quote  ✡️ ✡️ ✡️ ✡️ ✡️\n %s' % a)